## Goals:
    1. Take in DF's from 'filtered' (using pandas)
    2. Read first row, get first item in species column
    3. Use species[1] to look up sequence in fasta file
    4. User first row, raw position # to get nucleotide position + n (length of motif)
    5. Put this nucleotide sequence into new column

Questions:
  - align_position is the index in the alignment file
  - what format do you want the motif in?
      Seq() format or string?
  - I don't know which motif file to use or how to use it

Account for negative string reading GGATTA
                                    CCTAAT
                                    
-bcd:
- GGATTA positive
- TAATCC negative

In [1]:
from Bio import SeqIO
from Bio import Seq
import pandas as pd
import numpy as np
import csv
import re
import os

In [18]:
def seqToString(motif):
    """
    motif: a Seq that represents the motif 
    
    Returns the String representation of the motif
    """
    i = 0
    string = ""
    length = motif.__len__()
    while i < length:
        string += motif.__getitem__(i)
        i += 1
    return string

In [3]:
#needs to be fixed
def getMotifLength(motif_key):
    return 6

In [15]:
def getMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Creates a reader representing the data frame and iterates through each line retrieving the species ID and key.
    Parses through the alignment file to find the motif sequence.
    Returns an array of the motifs
    """
    f = open("../data/output/bcd/" + csv_file, 'r')
    #f = open(csv_file, 'r')
    data = csv.reader(f)
    line0 = next(data)
    motifs = []
    for line in data:
        species = line[2]
        index = int(float(line[5]))
        key = species.split('|')[0]
        record_dict = SeqIO.to_dict(SeqIO.parse("./data/alignments/align_outlier_rm_with_length_" + key + ".fa", "fasta"))
        sequence = record_dict[species]
        motif_length = getMotifLength(key);
        motif = sequence[index:index + motif_length]
        string_motif = seqToString(motif)
        motifs += [string_motif]
    f.close()
    motif_df = pd.DataFrame({'motif sequences': motifs})
    return motifs

In [5]:
def csvToDataFrame(csv_file):
    df = pd.read_csv(csv_file);
    return df

In [6]:
def csvToArray(csv_file):
    f = open(csv_file, 'r')
    data = csv.reader(f)
    labels = next(data)
    labels.append('motif sequence')
    array = []
    array.append(labels)
    
    for row in data:
        array.append(row)
    return array

In [7]:
def addMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Appends the DF of motif sequences as a column onto the filtered DF.
    """
    motif_seq = getMotifStrings(csv_file)
    df = csvToDataFrame(csv_file)
    df['motif sequences'] = motif_seq
    #os.path.join("/newDFs",df.to_csv("motif+" + csv_file)) 
    new_csv = df.to_csv("../data/output/2.motif_extraction_otput/motif+" + csv_file)
#     with open('/newDFs','w') as csvFile:
#         writer = csv.writer(new_csv)
#         writer.writerows(csvstff)
    return df

In [17]:
d = addMotifStrings("occurance_align_outlier_rm_with_length_VT11048.fa.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/alignments/align_outlier_rm_with_length_VT11048.fa'

In [42]:
print(d)

   Unnamed: 0      score                   species  raw_position    strand  \
0           0  11.612828  VT11048|0|MEMB005D|-|287            39  negative   
1           1  11.612828  VT11048|0|MEMB006A|+|286            38  negative   

   align_position                motif motif sequences  
0              60  MA0212.1_bcd.jaspar          GGATTA  
1              60  MA0212.1_bcd.jaspar          GGATTA  


In [43]:
for file in os.listdir("../TFBS_presence/data/output/bcd"):
    filename = os.path.basename(file)
    if filename.endswith(".csv"):
        addMotifStrings(str(filename))

FileNotFoundError: [Errno 2] No such file or directory: 'occurance_align_outlier_rm_with_length_VT16679.fa.csv'